In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import datetime

df_stikhi = pd.read_csv('./110920_poetry.csv', index_col = 0)
df_manifest = pd.read_csv('./110920_man.csv', index_col = 0)
df_dnevnik = pd.read_csv('./110920_dnevnik.csv', index_col = 0)
df_roman = pd.read_csv('./110920_roman.csv', index_col = 0)
df_gazet = pd.read_csv('./110920_man.csv', index_col = 0)

In [25]:
passw = ''
engine = sqlalchemy.create_engine('mysql+pymysql://madkehl:' + passw + '@127.0.0.1/prozhito_orig', encoding = 'utf-8')
connection = engine.connect()
metadata = sqlalchemy.MetaData()
notes = sqlalchemy.Table('notes', metadata, autoload = True, autoload_with = engine)
persons = sqlalchemy.Table('persons', metadata, autoload = True, autoload_with = engine)
diary =sqlalchemy.Table('diary', metadata, autoload = True, autoload_with = engine)

def execute(query):
    ResultProxy = connection.execute(query)
    return ResultProxy

def retrieve_notes(id_num):
    '''
    SELECT COUNT(YEAR(date)) AS NEWYEAR from notes WHERE YEAR(date) in (1901, 1903, 1906) GROUP BY YEAR(date); 
    **for some reason the year column is less complete than date column
    '''
    trial = execute('SELECT * FROM notes WHERE id = '+ str(id_num) + ';')
    ResultSet = trial.fetchall()
    return ResultSet



def resultToBio(diarynum):
    trans = execute('SELECT person FROM diary WHERE id = ' + str(diarynum) + ';')
    trans = trans.fetchall()
    if len(trans) < 1:
        return([[None, None, None]])
    bio_info = execute('SELECT firstName, lastName, info FROM persons WHERE id = ' + str(trans[0][0]) + ';')
    ResultSet = bio_info.fetchall()
    koshka_list = []
    for n in ResultSet:
        koshka_list.append([n[0], n[1], n[2]])
    return koshka_list


def sample(df, num_samples):
    dice = range(max(df['new_entry_type'])+ 1)
    for_tatyana = []
    id_col = []
    for i in dice:
        temp = df[df['new_entry_type'] == i]
        name = temp['new_name_type'].iloc[0]
        res = temp['entry_id'].sample(n=num_samples, random_state=7)
        res = res.reset_index(drop = True)
        for_tatyana.append(res)
        for i in range(num_samples):
            id_col.append(name)
    
    return(for_tatyana, id_col)
    

C:\Users\madke\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


In [47]:
df_dnevnik

,diarynum,entry_id,entry_year,new_entry_type,new_name_type
0,2,1506,1919,4,Spirit
1,2,1504,1919,4,Spirit
2,2,1497,1919,0,Routine
3,2,1488,1919,3,Literary
4,2,1488,1919,4,Spirit
...,...,...,...,...,...
12014,3275,463991,1908,2,Interpersonal
12015,3275,464004,1909,4,Spirit
12016,3275,464008,1909,2,Interpersonal
12017,3336,466611,1990,0,Routine


In [27]:
def full_path_to_entries(df, n):
    full_entries = []
    ids, entry_types = sample(df, n)
    for id_ in ids:
        for n in id_:
            full_entries.append(retrieve_notes(n))
            
    entry_id = []
    diarynum = []
    text = []
    year = []

    for entry in full_entries:
        for n in entry:
            entry_id.append(n[0])
            diarynum.append(n[1])
            text.append(n[2])
            year.append(n[3].year)
            
    bios = []
    for num in diarynum:
        bios.append(resultToBio(num))
        
    first_name = []
    last_name = []
    info = []

    for bio in bios:
        for n in bio:
            first_name.append(n[0])
            last_name.append(n[1])
            info.append(n[2])
            
    
    full_entries_df = pd.DataFrame({

        'name_type': entry_types,
        'entry_id': entry_id,
        'diarynum': diarynum,
        'text': text,
        'year': year,
        'first_name': first_name,
        'last_name': last_name,
        'info': info


    })
    
    return(full_entries_df)


In [6]:
def print_answers(df):
    for n in df.iterrows():
        i = n[1]

        print(i['name_type'])
        print(i['entry_id'])
        print(i['diarynum'])
        print()
        print(i['year'])
        print(str(i['first_name']) + ' ' + str(i['last_name']))
        print()
        print(i['text'])
        print()   

In [36]:
full_entries_df = full_path_to_entries(df_gazet, 10)

In [37]:
print_answers(full_entries_df)

Form
288276
888

2005
Сергей Есин

Суббота. В.С. сварила замечательные кислые щи из рыночной квашеной капусты, я их даже перелил в термос, но высунул нос на улицу, и на дачу все же не поехали — слишком холодно.    
Содержательно и хорошо провел день дома, все же эти пять-шесть часов, которые приходится тратить на поездку, на то, чтобы «запустить» дом, это часы из жизни. Их-то всегда и не хватает. Капитально убрался дома, занимался дневником, с удовольствием варил и тушил кролика, который куплен в Обнинске с месяц назад и все время находился в заточении в холодильнике, тушил я его и готовил под белым соусом.    
Но главное, дочитал книгу Умберто Эко. Та основная часть, ради которой все эти лекции были написаны и прочитаны, тот импульс ко всей работе, который я искал и не находил, наконец-то нашелся в последней, шестой, главе-лекции. В стержне любой книги обязательно есть какое-нибудь служение. Оно есть и здесь. Последняя глава вся посвящена вещи очевидной — «Протоколам сионских мудрецов